<a href="https://colab.research.google.com/github/WakWakBird/MLS/blob/main/lab-12-3-char-seq-softmax-only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow with GPU

This notebook provides an introduction to computing on a [GPU](https://cloud.google.com/gpu) in Colab. In this notebook you will connect to a GPU, and then run some basic TensorFlow operations on both the CPU and a GPU, observing the speedup provided by using the GPU.


## Enabling and testing the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

Next, we'll confirm that we can connect to the GPU with tensorflow:

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow 2.x selected.
Found GPU at: /device:GPU:0


## Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.862475891000031
GPU (s):
0.10837535100017703
GPU speedup over CPU: 35x


# Lab


In [25]:
import tensorflow as tf
import numpy as np

# 재현성을 위해 시드 고정
tf.random.set_seed(777)

# -----------------------------
# 1) 데이터 준비
# -----------------------------
sample = " if you want you"
chars = sorted(list(set(sample)))  # 고유 문자 집합 (정렬해서 순서 고정)
char2idx = {c: i for i, c in enumerate(chars)}
idx2char = {i: c for i, c in enumerate(chars)}

# 하이퍼파라미터
dic_size = len(chars)            # 문자 종류 수 (input, output 차원 동일)
sequence_length = len(sample) - 1
learning_rate = 0.1
batch_size = 1

# 인덱스 변환
sample_idx = [char2idx[c] for c in sample]
x_data = [sample_idx[:-1]]  # " if you want yo"
y_data = [sample_idx[1:]]   # "if you want you"

# -----------------------------
# 2) One-hot 인코딩
# -----------------------------
X_one_hot = tf.one_hot(x_data, depth=dic_size)        # (1, 15, dic_size)
Y_labels = np.array(y_data)                           # (1, 15)

# -----------------------------
# 3) 모델 정의 (Softmax만 사용)
# -----------------------------
model = tf.keras.Sequential([
    tf.keras.layers.Reshape((sequence_length, dic_size), input_shape=(sequence_length, dic_size)),
    tf.keras.layers.Dense(dic_size, activation='softmax')  # 순환 없이 Softmax 분류기
])

model.summary()
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              metrics=['accuracy'])

# -----------------------------
# 4) 학습
# -----------------------------
for epoch in range(3000):
    loss, acc = model.train_on_batch(X_one_hot, Y_labels)
    if (epoch+1) % 500 == 0:
        preds = model.predict(X_one_hot)
        result = np.argmax(preds, axis=2)
        result_str = ''.join(idx2char[c] for c in result[0])
        print(f"Epoch {epoch+1}, Loss: {loss:.4f}, Prediction: {result_str}")

# -----------------------------
# 5) 최종 결과
# -----------------------------
preds = model.predict(X_one_hot)
final_result = ''.join(idx2char[c] for c in np.argmax(preds, axis=2)[0])
print("\nFinal Prediction:", final_result)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape (Reshape)               │ (None, 15, 10)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 15, 10)         │           110 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 110 (440.00 B)

 Trainable params: 110 (440.00 B)

 Non-trainable params: 0 (0.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Epoch 500, Loss: 0.3207, Prediction: yf you yant you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Epoch 1000, Loss: 0.2994, Prediction: yf you yant you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Epoch 1500, Loss: 0.2921, Prediction: yf you yant you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Epoch 2000, Loss: 0.2885, Prediction: yf you yant you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Epoch 2500, Loss: 0.2862, Prediction: yf you yant you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Epoch 3000, Loss: 0.2848, Prediction: yf you yant you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step

Final Prediction: yf you yant you
